In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%tensorflow_version 1.x #use tensorflow magic to use version 1.x in colab

TensorFlow 1.x selected.


In [ ]:
#imports
import pandas as pd
import numpy as np
import nltk
import io
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from keras.preprocessing import text as keras_text, sequence as keras_seq
from keras.utils import np_utils
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.layers import Dense, Activation, Convolution1D, MaxPooling1D, Dropout, Flatten
from keras.models import Sequential, save_model
from sklearn.model_selection import train_test_split
from keras.layers import Embedding
from sklearn.metrics import classification_report,confusion_matrix

Using TensorFlow backend.


In [ ]:
df_train=pd.read_csv("/content/drive/My Drive/minor/english_dataset/english_dataset_prepro.csv")

In [ ]:
df_train.head()

,text_id,text,task_1,task_2,task_3
0,hasoc_en_1,dhonikeepstheglove watch sports minister kiren...,NOT,NONE,NONE
1,hasoc_en_2,politico remember clearly individual1 admitted...,HOF,HATE,TIN
2,hasoc_en_3,cricketworldcup guess would winner cwc19 team ...,NOT,NONE,NONE
3,hasoc_en_4,corbyn politically intellectual borisjohnsonsh...,NOT,NONE,NONE
4,hasoc_en_5,best teamindia another swimming competition su...,NOT,NONE,NONE


In [ ]:
Column_Sequence=np.array(df_train["text"],dtype="str") #storing value  of tweets or text in numpy array of strings

In [ ]:
Tokenizer=RegexpTokenizer(r"\w+[a-z]+") #create a reg exp tokenizer object for text to make a list of strings and lowercase text (earlier also removed stopwords but that has been moved to preprocessing code)

In [ ]:
fresh=[]
for ix in Column_Sequence:
    sen=ix.lower()                        #lower case
    sen=Tokenizer.tokenize(sen)           #tokenize the sentences.
    wordss=[(Word) for Word in sen]
    sentance=" ".join(wordss)
    fresh.append(sentance)                #append all in one.

In [ ]:
tokenizer = keras_text.Tokenizer(char_level = False,lower=True) #create a kerastokenizer object
t=tokenizer.fit_on_texts(list(fresh))                           #fit on training text
list_tokenized_train = tokenizer.texts_to_sequences(fresh)
X_t = keras_seq.pad_sequences(list_tokenized_train, maxlen=300,padding="post") #maxlen after padding 300
word_index = tokenizer.word_index      #get a list of all tokens or words in the vocab

In [ ]:
#extract word2vec vectors for words in vocab
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

from gensim.models.keyedvectors import KeyedVectors
#load word2vec vectors from file.
word_vectors = KeyedVectors.load_word2vec_format('/content/drive/My Drive/minor/wordtovec/GoogleNews-vectors-negative300.bin.gz', binary=True)

EMBEDDING_DIM=300
vocabulary_size=len(word_index)+1
embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))
for word, i in word_index.items():
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector            #store the vector if found.
    except KeyError:
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)  #store a rndom vector if not found

del(word_vectors)  #del word2vec vectors to free memory

from keras.layers import Embedding
#create an embediing layer from word2vec vectors of our vocab
embedding_layer = Embedding(vocabulary_size,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            trainable=False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
len(tokenizer.word_counts)

16526

In [ ]:
X_train=X_t      #training text

In [ ]:
X_train.shape

(5852, 300)

In [ ]:
Y=df_train["task_1"]   #labels of classes with strings.

In [ ]:
Y = Y.map({'HOF':0, 'NOT': 1})  #map the class names to numbers.

In [ ]:
# One-hot encode label
Y_train=np_utils.to_categorical(Y)

In [ ]:
Y_train.shape

(5852, 2)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_train,Y_train, test_size=0.10, random_state=42) #training validation split

In [ ]:
y_train.shape

(5266, 2)

In [ ]:
x_train.shape

(5266, 300)

In [ ]:
max_len=300   #length of text sequence

In [ ]:
x_train

array([[    4,    21,   188, ...,     0,     0,     0],
       [ 1495,  1247,    70, ...,     0,     0,     0],
       [ 9400,  9401,  2422, ...,     0,     0,     0],
       ...,
       [ 3805,  2826, 15341, ...,     0,     0,     0],
       [  665, 15679,   968, ...,     0,     0,     0],
       [  382,  3152,  8291, ...,     0,     0,     0]], dtype=int32)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Input , Layer
from keras.layers import Embedding, GRU, Bidirectional , Reshape, Dot
from keras.models import Sequential, Model
from keras.layers.merge import concatenate
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Input
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras import metrics
from keras import backend as K
from keras.regularizers import l2
import tensorflow as tf
from keras import initializers ,constraints, regularizers

In [ ]:
#implementation of attention layer in keras
def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)
    

class AttentionWithContext(Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    Note: The layer has been tested with Keras 2.0.6
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """

    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight(shape=(input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight(shape=(input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

In [ ]:
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding, CuDNNGRU
from keras.models import Model
from keras.initializers import Constant
from keras.callbacks import ModelCheckpoint
#BiGRU+ATTENTION
sequence_input2 = Input(shape=(max_len,), dtype='int32') #input layer
embedded_sequences2 = embedding_layer(sequence_input2)
gru1=Bidirectional(CuDNNGRU(128, return_sequences=True))(embedded_sequences2) #bidirectional gru layer optimized for gpu
gru2= Bidirectional(CuDNNGRU(128, return_sequences=True))(gru1)
attn_outs=AttentionWithContext()(gru2) #attention layer
l_dense2 = Dense(128, activation='relu')(attn_outs)
preds2 = Dense(2, activation='softmax')(l_dense2) #ouput layer

model2 = Model(sequence_input2, preds2)
model2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

print("Bi-GRU+ATTENTION")
model2.summary()
cp2=ModelCheckpoint('model_gru_attn.hdf5',monitor='val_acc',verbose=1,save_best_only=True)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
GRU
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 300)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 300, 300)          4958100   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 300, 256)          330240    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 300, 256)          296448    
_________________________________________________________________
attention_with_context_1 (At (None, 256)               66048     
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
________________________________________

In [ ]:
from keras.layers import Dense, Input, GlobalMaxPooling1D, Bidirectional
from keras.layers import Conv1D, MaxPooling1D, Embedding, CuDNNLSTM
from keras.models import Model
from keras.initializers import Constant
from keras.callbacks import ModelCheckpoint
#BI-LSTM+ATTENTION
sequence_input3 = Input(shape=(max_len,), dtype='int32') #input layer
embedded_sequences3 = embedding_layer(sequence_input3)
lstm1 = Bidirectional(CuDNNLSTM(128, return_sequences=True))(embedded_sequences3) #bi-directional lstm layer optimized for gpu
lstm2= Bidirectional(CuDNNLSTM(128, return_sequences=True))(lstm1)
attn_outs3=AttentionWithContext()(lstm2) #attention layer
l_dense3 = Dense(128, activation='relu')(attn_outs3)
preds3 = Dense(2, activation='softmax')(l_dense3) #output layer

model3 = Model(sequence_input3, preds3)
model3.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

print("Bi-LSTM+Attention")
model3.summary()
cp3=ModelCheckpoint('model_lstm_attn.hdf5',monitor='val_acc',verbose=1,save_best_only=True)

LSTM
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 300)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 300, 300)          4958100   
_________________________________________________________________
bidirectional_3 (Bidirection (None, 300, 256)          440320    
_________________________________________________________________
bidirectional_4 (Bidirection (None, 300, 256)          395264    
_________________________________________________________________
attention_with_context_2 (At (None, 256)               66048     
_________________________________________________________________
dense_3 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_4 (Dense)              (None, 2)                

In [ ]:
history2=model2.fit(x_train, y_train, validation_data=(x_test, y_test),epochs=7, batch_size=32,callbacks=[cp2]) #train bigru with attention with checkpoint


Train on 5266 samples, validate on 586 samples
Epoch 1/7
5266/5266 [==============================] - 50s 9ms/step - loss: 0.6578 - acc: 0.6215 - val_loss: 0.6200 - val_acc: 0.6758

Epoch 00001: val_acc improved from -inf to 0.67577, saving model to model_gru_attn.hdf5
Epoch 2/7
5266/5266 [==============================] - 44s 8ms/step - loss: 0.5830 - acc: 0.6943 - val_loss: 0.5745 - val_acc: 0.6928

Epoch 00002: val_acc improved from 0.67577 to 0.69283, saving model to model_gru_attn.hdf5
Epoch 3/7
5266/5266 [==============================] - 43s 8ms/step - loss: 0.4487 - acc: 0.8035 - val_loss: 0.6471 - val_acc: 0.6860

Epoch 00003: val_acc did not improve from 0.69283
Epoch 4/7
5266/5266 [==============================] - 43s 8ms/step - loss: 0.2696 - acc: 0.8919 - val_loss: 0.7606 - val_acc: 0.6468

Epoch 00004: val_acc did not improve from 0.69283
Epoch 5/7
5266/5266 [==============================] - 41s 8ms/step - loss: 0.1515 - acc: 0.9432 - val_loss: 0.9597 - val_acc: 0.6672

In [ ]:
history3=model3.fit(x_train, y_train, validation_data=(x_test, y_test),epochs=7, batch_size=32,callbacks=[cp3]) #train bilstm with attention with checkpoint

Train on 5266 samples, validate on 586 samples
Epoch 1/7
5266/5266 [==============================] - 41s 8ms/step - loss: 0.2449 - acc: 0.8965 - val_loss: 0.9824 - val_acc: 0.6604

Epoch 00001: val_acc improved from -inf to 0.66041, saving model to model_lstm_attn.hdf5
Epoch 2/7
5266/5266 [==============================] - 40s 8ms/step - loss: 0.1055 - acc: 0.9605 - val_loss: 1.4241 - val_acc: 0.6451

Epoch 00002: val_acc did not improve from 0.66041
Epoch 3/7
5266/5266 [==============================] - 40s 8ms/step - loss: 0.0689 - acc: 0.9772 - val_loss: 1.3254 - val_acc: 0.6399

Epoch 00003: val_acc did not improve from 0.66041
Epoch 4/7
5266/5266 [==============================] - 40s 8ms/step - loss: 0.0456 - acc: 0.9821 - val_loss: 1.7216 - val_acc: 0.6365

Epoch 00004: val_acc did not improve from 0.66041
Epoch 5/7
5266/5266 [==============================] - 40s 8ms/step - loss: 0.0434 - acc: 0.9820 - val_loss: 1.5168 - val_acc: 0.6331

Epoch 00005: val_acc did not improve fr

In [ ]:
w2=model2.predict(x_test) #predict on testing data for bi-gru with attention
w2=np.argmax(w2,axis=1)
w3=model3.predict(x_test) #predict on testing data for bi-lstm with attention
w3=np.argmax(w3,axis=1)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
#true labels for validation data
Y_actual=[]
for ix in y_test:
    Y_actual.append(np.argmax(ix))

In [ ]:
print("Bi-GRU+ATTN")
print(classification_report(Y_actual,w2))
print("Bi-LSTM+ATTN")
print(classification_report(Y_actual,w3))

GRU+ATTN
              precision    recall  f1-score   support

           0       0.55      0.46      0.50       230
           1       0.68      0.76      0.72       356

    accuracy                           0.64       586
   macro avg       0.62      0.61      0.61       586
weighted avg       0.63      0.64      0.63       586

LSTM+ATTN
              precision    recall  f1-score   support

           0       0.53      0.55      0.54       230
           1       0.70      0.69      0.70       356

    accuracy                           0.63       586
   macro avg       0.62      0.62      0.62       586
weighted avg       0.64      0.63      0.64       586



In [ ]:
#testing data
data_test=pd.read_csv('/content/drive/My Drive/minor/english_dataset/hasoc2019_en_test-2919_prepro.csv')

In [ ]:
data_test.head()

,text_id,text,task_1,task_2,task_3
0,hasoc_en_902,west bengal doctor crisis protesting doctors a...,NOT,NONE,NONE
1,hasoc_en_416,68 5 million people forced leave homes read,NOT,NONE,NONE
2,hasoc_en_207,came saw look fort good luck,NOT,NONE,NONE
3,hasoc_en_595,get brexit delivered october 31st help build m...,NOT,NONE,NONE
4,hasoc_en_568,fuck go back dark ages cow ibnliverealtime rap...,HOF,PRFN,UNT


In [ ]:
data_test.text=data_test.text.astype(str)
df_text=data_test['text']

In [ ]:
Column_Sequence=np.array(data_test["text"],dtype="str")
fresh=[]
for ix in Column_Sequence:
    sen=ix.lower()
    sen=Tokenizer.tokenize(sen)    #use the reg exp tokenizer object for text to make a list of strings and lowercase text (earlier also removed stopwords but that has been moved to preprocessing code)
    wordss=[(Word) for Word in sen]
    sentance=" ".join(wordss)
    fresh.append(sentance)

In [ ]:
tokenizer = keras_text.Tokenizer(char_level = False,lower=True)  #create a kerastokenizer object
t=tokenizer.fit_on_texts(list(fresh))
list_tokenized_train = tokenizer.texts_to_sequences(fresh)
X_test = keras_seq.pad_sequences(list_tokenized_train, maxlen=300,padding="post") #maxlen padding 300

In [ ]:
w_test2=model2.predict(X_test)  #predict class of testing data for bi-gru+attn
w_test2=np.argmax(w_test2,axis=1)
w_test3=model3.predict(X_test)  #predict class of testing data for bi-lstm+attn
w_test3=np.argmax(w_test3,axis=1)

In [ ]:
data_test['task_1'] = data_test['task_1'].map({'HOF': 0, 'NOT': 1}) #map label strings to numbers

In [ ]:
y_test_actual=data_test['task_1']

In [ ]:
print("Bi-GRU+ATTN")
print(classification_report(y_test_actual,w_test2))
print("Bi-LSTM+ATTN")
print(classification_report(y_test_actual,w_test3))

GRU+ATTN
              precision    recall  f1-score   support

           0       0.23      0.24      0.23       288
           1       0.74      0.73      0.74       865

    accuracy                           0.61      1153
   macro avg       0.48      0.48      0.48      1153
weighted avg       0.61      0.61      0.61      1153

LSTM+ATTN
              precision    recall  f1-score   support

           0       0.25      0.32      0.28       288
           1       0.75      0.68      0.71       865

    accuracy                           0.59      1153
   macro avg       0.50      0.50      0.49      1153
weighted avg       0.62      0.59      0.60      1153

